In [ ]:
access_token = ""

In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score # Import all needed metrics
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, default_data_collator

/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


ImportError: Unable to import required dependencies:
numpy: No module named 'numpy'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

In [ ]:
# Load the pre-trained BERT model and tokenizer
#model_name = "mistralai/Mistral-7B-v0.1"
model_name = "/home/patrick.araujo/llama2/llama/llama-2-7b-hf"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, token=access_token)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = 'right'

In [ ]:
dataset = pd.read_csv('/home/patrick.araujo/llama2/datasets/balanced_output_train.csv')

In [ ]:
dataset

In [ ]:
dataset.shape

In [ ]:
# Create train and validation datasets
comments = dataset[['content', 'sentiment']]
train_data, val_data = train_test_split(comments, test_size=0.2, random_state=42)

In [ ]:
def tokenizerFunction(string):
    tokens = tokenizer(string, padding=True, truncation=True)
    return tokens

train_encodings = train_data['content'].apply(tokenizerFunction)
val_encodings = val_data['content'].apply(tokenizerFunction)

In [ ]:
train_encodings

In [ ]:
val_encodings

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long) # Ensure labels are of type long
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Create datasets
train_labels = train_data['sentiment'].tolist()
val_labels = val_data['sentiment'].tolist()

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [ ]:
# Define training arguments with early stopping
training_args = TrainingArguments(
    # Positional arguments:
    output_dir="./llama2_sentiment_model",
    logging_dir="./logs",

    # Keyword arguments:
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
    num_train_epochs=3,
    save_steps=500,
    metric_for_best_model="eval_loss",
)

In [ ]:
def compute_metrics(p):
  pred, labels = p
  pred = np.argmax(pred, axis=1)
  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred, average='macro')
  precision = precision_score(y_true=labels, y_pred=pred, average='macro')
  f1 = f1_score(y_true=labels, y_pred=pred, average='macro')  # Add line for F1 score
  return  {
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'f1': f1  # Include F1 score in the dictionary
          }

In [ ]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
x = train_data['sentiment'labels.values

In [ ]:
x

In [ ]:
# Correct approach to combine the data
combined_data = []
for (index, (input_ids, attention_mask)), label in zip(train_df.items(), x):
    combined_data.append([input_ids, attention_mask, label])

# Convert combined_data to a pandas Series or DataFrame as needed
combined_series = pd.Series(combined_data, index=train_df.index)

In [ ]:
# Assuming your Series is named `df`
def add_label(row):
    # Define the logic for calculating the label based on your specific needs
    # For example, if you have separate label data:
    label = your_label_data[row.name]
    return [row[0], row[1], label]

df = train_df.apply(add_label, axis=1)

In [ ]:
# Tokenize train and validation data
train_tokens = tokenizer(train_data['content']), padding=True, truncation=True)
val_tokens = tokenizer(str(val_data['content']), padding=True, truncation=True)

# Access tokenized text and other attributes
train_text = train_tokens['input_ids']
train_attention_mask = train_tokens['attention_mask']
# Create a DataFrame from the list of dictionaries
train_df = pd.DataFrame(inputID_attentionMask(train_text, train_attention_mask))
train_df['labels'] = train_data['sentiment']
print(train_df)
# Do the same for validation data
val_text = val_tokens['input_ids']
val_attention_mask = val_tokens['attention_mask']





In [ ]:
# Define batch size and create data loaders
batch_size = 8  # Adjust as needed
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=default_data_collator)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=default_data_collator)

In [ ]:
# Define training arguments with early stopping
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir="./llama2_model",
    evaluation_strategy="steps",
    eval_steps=100,  # Adjust as needed
    save_total_limit=2,  # Adjust as needed. Only last 2 models are saved. Older ones are deleted.
    num_train_epochs=3,  # Adjust as needed
    save_steps=500,  # Adjust as needed
    logging_dir="./logs",
    metric_for_best_model="eval_loss",  # Use validation loss for early stopping
    load_best_model_at_end=True
)

In [ ]:
def compute_metrics(p):
  pred, labels = p
  pred = np.argmax(pred, axis=1)
  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred, average='macro')
  precision = precision_score(y_true=labels, y_pred=pred, average='macro')
  f1 = f1_score(y_true=labels, y_pred=pred, average='macro')  # Add line for F1 score
  return  {Z
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'f1': f1  # Include F1 score in the dictionary
          }

In [ ]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
peft_config = LoraConfig(
        lora_alpha=16, 
        lora_dropout=0.1,
        r=64,
        bias="none",
        target_modules="all-linear",
        task_type=TaskType.SEQ_CLS,
)

training_arguments = TrainingArguments(
    output_dir="logs",                        # directory to save and repository id
    num_train_epochs=3,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)